In [1]:
import pandas as pd 

#Enter desired years of data
YEARS = [2019,2018,2017]

pbp = pd.DataFrame()

for i in YEARS:  
    #low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    #sort=True eliminates a warning and alphabetically sorts columns
    pbp = pbp.append(i_data, sort=True)

#Give each row a unique index
pbp.reset_index(drop=True, inplace=True)

In [43]:
# first set of model mutations - https://github.com/guga31bb/nflfastR-data/blob/106adcf0033fa741fe8be11a2d52232aecc26a5e/models/train_xyac_model.R#L27
model_data = pbp[(pbp.season >= 2006) & (pbp.complete_pass == 1) & (pbp.yards_after_catch.notnull()) &
         (pbp.yards_after_catch >= -20) & (pbp.air_yards < pbp.yardline_100)]

In [44]:
# new columns https://github.com/guga31bb/nflfastR-data/blob/106adcf0033fa741fe8be11a2d52232aecc26a5e/models/train_xyac_model.R#L30
import regex # regex that supports variable-length positive lookbehind https://bitbucket.org/mrabarnett/mrab-regex/src/hg/
import math

model_data["distance_to_goal"] = model_data.yardline_100 - model_data.air_yards
model_data["pass_middle"] = model_data.pass_location.apply(lambda x: 1 if (x == 'middle') else 0)
model_data["air_is_zero"] = model_data.air_yards.apply(lambda x: 1 if (x == 0) else 0)
model_data["distance_to_sticks"] = model_data.air_yards - model_data.ydstogo
model_data["yards_after_catch"] = model_data.yards_after_catch.clip(-5, 70)
model_data["receiver_player_name"] = model_data.desc.apply(lambda x: regex.search("(?<=((to)|(for))\\s[0-9]{0,2}\\-{0,1})[A-Z][A-z]*\\.\\s?[A-Z][A-z]+(\\s(I{2,3})|(IV))?", x))
# note: receiver_player_name contains regex.Match objects, not strings
model_data["down1"] = model_data.down.apply(lambda x: 1 if (x == 1) else 0)
model_data["down2"] = model_data.down.apply(lambda x: 1 if (x == 2) else 0)
model_data["down3"] = model_data.down.apply(lambda x: 1 if (x == 3) else 0)
model_data["down4"] = model_data.down.apply(lambda x: 1 if (x == 4) else 0)

# from docs: 1 - 2006-2013, 2 - 2014-2017, 3 - 2018 and beyond
model_data["era1"] = model_data.season.apply(lambda x: 1 if (x < 2006) else 0)
model_data["era2"] = model_data.season.apply(lambda x: 1 if (x >= 2006 and x <= 2013) else 0)
model_data["era3"] = model_data.season.apply(lambda x: 1 if (x >= 2014 and x <= 2017) else 0)
model_data["era4"] = model_data.season.apply(lambda x: 1 if (x >= 2018) else 0)

# roof: "outdoors", "retractable", "dome"; values: ['dome', 'outdoors', 'closed', 'open']
model_data["dome"] = model_data.roof.apply(lambda x: 1 if (x == 'dome') else 0)
model_data["outdoors"] = model_data.roof.apply(lambda x: 1 if (x == 'outdoors') else 0)
model_data["retractable"] = model_data.roof.apply(lambda x: 1 if (x == 'closed' or x == 'open') else 0)

# "home" -- assuming that offense is home
model_data["home"] = model_data.apply(lambda x: 1 if (x.posteam == x.home_team) else 0, axis=1)

# checking pass validity: https://github.com/mrcaseb/nflfastR/blob/master/R/helper_add_cp_cpoe.R#L64
def check_pass_validity(row):
    return ((row.complete_pass == 1) | (row.incomplete_pass == 1) | (row.interception == 1)) & (~math.isnan(row.air_yards)) & (row.air_yards >= -15) & (row.air_yards < 70) & (row.receiver_player_name != None) & (row.pass_location != None)

model_data["valid_pass"] = model_data.apply(lambda x: check_pass_validity(x), axis=1)

model_data.head()

/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,aborted_play,air_epa,air_wpa,air_yards,assist_tackle,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,...,down4,era1,era2,era3,era4,dome,outdoors,retractable,home,valid_pass
7,0,-0.553484,0.000000,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,1,1
8,0,0.974937,0.013921,11.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,1,1
15,0,-0.544798,-0.015032,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,1,1
22,0,-0.738814,-0.002456,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,0,1
25,0,-0.252091,-0.003843,5.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,0,1


In [40]:
model_data.home_team

7         MIN
8         MIN
15        MIN
22        MIN
25        MIN
         ... 
143881     NE
143884     NE
143897     NE
143899     NE
143900     NE
Name: home_team, Length: 33821, dtype: object

In [38]:
prepare_cp_data = model_data[["season", "complete_pass", "air_yards", "yardline_100", "ydstogo",
    "down1", "down2", "down3", "down4", "air_is_zero", "pass_middle",
    "era2", "era3", "era4", "qb_hit", "home",
    "outdoors", "retractable", "dome", "distance_to_sticks"]]

prepare_cp_data.head()

# nrounds <- 560
# params <-
#   list(
#     booster = "gbtree",
#     objective = "binary:logistic",
#     eval_metric = c("logloss"),
#     eta = 0.025,
#     gamma = 5,
#     subsample = 0.8,
#     colsample_bytree = 0.8,
#     max_depth = 4,
#     min_child_weight = 6,
#     base_score = mean(model_data$complete_pass)
#   )

# cv_results <- map_dfr(2006:2019, function(x) {
#   test_data <- model_data %>%
#     filter(season == x) %>%
#     select(-season)
#   train_data <- model_data %>%
#     filter(season != x) %>%
#     select(-season)

#   full_train <- xgboost::xgb.DMatrix(model.matrix(~ . + 0, data = train_data %>% select(-complete_pass)),
#     label = train_data$complete_pass
#   )
#   cp_model <- xgboost::xgboost(params = params, data = full_train, nrounds = nrounds, verbose = 2)

#   preds <- as.data.frame(
#     matrix(predict(cp_model, as.matrix(test_data %>% select(-complete_pass))))
#   ) %>%
#     dplyr::rename(cp = V1)

#   cv_data <- bind_cols(test_data, preds) %>% mutate(season = x)
#   return(cv_data)
# })

KeyError: "['era4', 'home'] not in index"